### 1. 모듈 경로 설정

In [ ]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

scripts_path = os.path.join(project_root, 'scripts')
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

yolov5_path = os.path.join(project_root, 'yolov5')
if yolov5_path not in sys.path:
    sys.path.append(yolov5_path)

from scripts.upload_to_hf import hf_login, upload_model_to_hf

### 2. 모델 파인튜닝

In [ ]:
# data.yaml 파일 경로 설정
data_yaml_path = '../yolov5/data/yolo/data.yaml'

# 모델 파인튜닝 실행
!python ../yolov5/train.py --img 640 --batch 32 --epochs 1000 --data {data_yaml_path} --weights yolov5s.pt --cache

In [ ]:
# data.yaml 파일 경로 설정
data_yaml_path = '../yolov5/data/coco/data.yaml'

# 모델 파인튜닝 실행
!python ../yolov5/train.py --img 640 --batch 32 --epochs 1000 --data {data_yaml_path} --weights ../yolov5/runs/train/exp2/weights/best.pt --cache

### 3. 허깅페이스에 업로드

In [ ]:
hf_login()

repo_name = "jspark2000/yolov5-vehicle"
local_model_dir = "../yolov5/runs/train/exp2/weights"
filenames = ["best.pt", "last.pt"]

upload_model_to_hf(repo_name, local_model_dir, filenames)

### 4. 허깅페이스에서 다운로드 후 테스트

In [ ]:
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
hf_path = 'https://huggingface.co/jspark2000/yolov5-vehicle/resolve/main/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=hf_path, force_reload=True)

image_url = "../yolov5/data/images/bus.jpg"
img = Image.open(image_url)

results = model(img)
results_df = results.pandas().xyxy[0]
results.show()

results_df.columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name']
print(results_df)